In [289]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')

import rows
import locale

print("Imported!")

Imported!


In [239]:
from pathlib import Path
prefeitos_path = Path('prefeitos-eleitos')

prefeitos = None

def get_state_name(filename):
    filename = str(filename)
    state_ext = filename.split(' - ')[-1:][0]
    state_ext = state_ext.split('/')[-1:][0]
    state = state_ext.split('.')[:-1][0]
    return state.capitalize()
    

with rows.locale_context(name='pt_BR.UTF-8', category=locale.LC_NUMERIC):
    for csv_filename in prefeitos_path.iterdir():
        if not csv_filename.is_file():
            continue
        with open(str(csv_filename), 'rb') as csv_file:
            if prefeitos is None:
                prefeitos = rows.import_from_csv(csv_file)
                prefeitos['state'] = [get_state_name(csv_filename)] * len(prefeitos)
            else:
                csv_tmp = rows.import_from_csv(csv_file)
                csv_tmp['state'] = [get_state_name(csv_filename)] * len(csv_tmp)
                for dic in rows.export_to_dicts(csv_tmp):
                    prefeitos.append(dic)

def segundo_turno(pref):
    return pref.split('* ')[-1:][0], bool(len(pref.split('* ')) - 1)

segundo_turno = [
    segundo_turno(prefeito)for prefeito in prefeitos['nome']
]
prefeitos['nome'] = [segundo[0] for segundo in segundo_turno]
prefeitos['segundo_turno'] = [segundo[1] for segundo in segundo_turno]

In [240]:
df = pd.DataFrame({field: prefeitos[field] for field in prefeitos.field_names})
df.head()

,municipio,nome,num,partidocoligacao,segundo_turno,state,validos,votacao
0,ABAÍRA,DIGA,14,PTB/PTB / PMDB / DEM / PTN / PRB,False,Bahia,0.513,2701
1,ABARÉ,FERNANDO TOLENTINO,13,PT/PT / PP / PHS / DEM / PTN / PR / PSL / PSD,False,Bahia,0.5108,5482
2,ACAJUTIBA,ALEX FREITAS,15,PMDB/PMDB / PSDB / PPL / SD / DEM,False,Bahia,0.5227,4482
3,ADUSTINA,PAULO SERGIO,17,PSL/PSL / PT / PTN / SD,False,Bahia,0.5186,4644
4,ÁGUA FRIA,MANOEL POTINHA,25,DEM/DEM / PDT / PMB / PV,False,Bahia,0.5113,4617


In [247]:
with open('consolidado-prefeitos.csv', 'wb') as f:
    f.write(rows.export_to_csv(prefeitos))


In [242]:
print(df.groupby('municipio').count())

                        nome  num  partidocoligacao  segundo_turno  state  \
municipio                                                                   
ABADIA DE GOIÁS            1    1                 1              1      1   
ABADIA DOS DOURADOS        1    1                 1              1      1   
ABADIÂNIA                  1    1                 1              1      1   
ABAETETUBA                 1    1                 1              1      1   
ABAETÉ                     1    1                 1              1      1   
ABAIARA                    1    1                 1              1      1   
ABARÉ                      1    1                 1              1      1   
ABATIÁ                     1    1                 1              1      1   
ABAÍRA                     1    1                 1              1      1   
ABDON BATISTA              1    1                 1              1      1   
ABEL FIGUEIREDO            1    1                 1              1      1   

In [246]:
count_municipios = df.groupby(['state', 'municipio']).count()
with open('municipios-brasil.csv', 'w') as f:
    f.write(count_municipios.to_csv())
print(count_municipios)

                                        nome  num  partidocoligacao  \
state     municipio                                                   
Acre      ACRELÂNDIA                       1    1                 1   
          ASSIS BRASIL                     1    1                 1   
          BRASILÉIA                        1    1                 1   
          BUJARI                           1    1                 1   
          CAPIXABA                         1    1                 1   
          CRUZEIRO DO SUL                  1    1                 1   
          EPITACIOLÂNDIA                   1    1                 1   
          FEIJÓ                            1    1                 1   
          JORDÃO                           1    1                 1   
          MANOEL URBANO                    1    1                 1   
          MARECHAL THAUMATURGO             1    1                 1   
          MÂNCIO LIMA                      1    1                 1   
      

In [248]:
with open('municipios-brasil.html', 'w') as f:
    f.write(count_municipios.to_html())

In [284]:
plt.close('all')

In [295]:
segundo_turno = df[df.segundo_turno == True]
group_segundo = segundo_turno.groupby('municipio')
segundo_turno.cumsum()
segundo_turno.plot.
